In [9]:
# Connection Strings - json

# Connection Strings - SQL database (where we are writing to)

# Translate json file into objects

# Fstrings to write dynamic SQL code into tables

# Close connection strings

In [84]:
# Import Depencies

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
#import numpy as np
import requests
from pprint import pprint
import time
import json
from requests.structures import CaseInsensitiveDict

#new libraries
import pyarrow #not sure this is necessary, but it probably is #pip install pyarrow

from apikey import apikey

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [25]:
#Section for user input

lat = ''
lon = ''

default = [0, 0]
germany = [51.1657, 10.4515]
london = [51.507351, -0.127758]
brisbane = [-27.469770, 153.025131]
cleveland = [41.4993, -81.6944]

location_search = [123456789,123456789] #user_defined or written to pre-determined

default_city_list =  {'Germany' : germany , 'London' : london, 'Brisbane' : brisbane, 'Cleveland' : cleveland}


print('Available list of cities\n')
for k, v in default_city_list.items():
    print(k)



Available list of cities

Germany
London
Brisbane
Cleveland


In [26]:
user_coordinates = input("Would you like pick from a list of available cities? yes or no")

if user_coordinates.lower() == 'y' or user_coordinates.lower() == 'yes':

    match = input("Please pick a city from the list.\n")

    for k, v in default_city_list.items():
        if match.lower() == k.lower():
            location_search = v

elif user_coordinates.lower() == 'n' or user_coordinates.lower() == 'no':
    user_lat = input("Please enter the latitude. ")
    user_lon = input("Please enter the longitude. ")
    location_search = [float(user_lat), float(user_lon)]

else:
    raise Exception("Invalid Input. Must specify Yes (y) or No (n)")

if location_search == [123456789, 123456789]:
    raise Exception("Invalid location. Use a name that is in the list, or choose No (n) your own coordinates.")
    
print(f'Using coordinates {location_search}\n')


Using coordinates [41.4993, -81.6944]



In [27]:



#get latitude, longitude, and radius for starting point - New York City is [40.712776 N, -74.005974 W], Germany is [51.1657 N, 10.4515 E]  Germany country code is "de"
#London is [51.507351, -0.127758] Brisbane is [-27.469770, 153.025131]
#I can find London, but US cities are not coming up





#THESE VALUES ARE JUST FOR TESTING. DELETE 
#location_search = germany
#lat = "-27.469770"
#lon = "153.025131"

#User inputs should be placed for these too - Just ask if the default values are OK.
radius_meters = 5000
country_code = ''
limit = 10
offset = 0 #default is zero

location_filter = f'circle:{location_search[1]},{location_search[0]},{radius_meters}'

if country_code != "":
    country_code = country_code + f"|countrycode:{country_code}"

#headers = CaseInsensitiveDict()
#headers["Accept"] = "application/json"

#resp = requests.get(url, headers=headers)

#print(resp.status_code)

In [28]:
# List of categories
# catering, commercial.food_and_drinks, accomodations, entertainment, leisure, parking, wheelchair
category_string = ""

activity = True
commercial = True
commercial_catering = False
commercial_supermarket = True
accomodations = False
entertainment = True
leisure = False
parking = False
wheelchair = False


In [29]:
# dictionary of each category that will be captured
# we also may want to include user input for this too. Loop through these, or other questions. Set value to True. Default value is false. Must have at least one of these active in order
# to run the script. If not, throw an error
# We either need to specify that the list needs to be under 500, because that is the max number, or create 
category_dict = {'activity' : activity, 'commercial' : commercial, 'commerical.catering' : commercial_catering, 'accomodation' : accomodations, \
                        'entertainment' : entertainment, 'leisure': leisure, 'parking' : parking, 'wheelchair.yes' : wheelchair}

string_of_categories = ""

for k,v in category_dict.items():
    if v:
        string_of_categories = string_of_categories + k + ','

#remove last comma
string_of_categories = string_of_categories[:-1]

# conditions, organic.only, vegetarian.only, wheelchair.yes
url2 = f"https://api.geoapify.com/v2/places?categories={string_of_categories}&filter={location_filter}&limit={limit}&offset={offset}&apiKey={apikey}"

# atms in radius, 

# contact information for the restaurants, opening/closing hours, web address, phone number

In [30]:
# get API dictionary
resp = requests.get(url2, headers=headers)

print(resp.status_code)

json_info = resp.json()
pprint(json_info)

200
{'features': [{'geometry': {'coordinates': [-81.69677670688294,
                                            41.50747877025738],
                            'type': 'Point'},
               'properties': {'address_line1': 'Great Lakes Science Center',
                              'address_line2': '601 Erieside Avenue, '
                                               'Cleveland, OH 44114, United '
                                               'States of America',
                              'categories': ['building',
                                             'building.tourism',
                                             'entertainment',
                                             'entertainment.museum',
                                             'internet_access',
                                             'wheelchair',
                                             'wheelchair.yes'],
                              'city': 'Cleveland',
                              'countr

In [31]:
pd.json_normalize(json_info["features"]).columns.values

array(['type', 'properties.name', 'properties.country',
       'properties.country_code', 'properties.state', 'properties.county',
       'properties.city', 'properties.postcode', 'properties.district',
       'properties.street', 'properties.housenumber', 'properties.lon',
       'properties.lat', 'properties.state_code', 'properties.formatted',
       'properties.address_line1', 'properties.address_line2',
       'properties.categories', 'properties.details',
       'properties.datasource.sourcename',
       'properties.datasource.attribution',
       'properties.datasource.license', 'properties.datasource.url',
       'properties.datasource.raw.ele', 'properties.datasource.raw.name',
       'properties.datasource.raw.phone',
       'properties.datasource.raw.museum',
       'properties.datasource.raw.osm_id',
       'properties.datasource.raw.tourism',
       'properties.datasource.raw.website',
       'properties.datasource.raw.building',
       'properties.datasource.raw.osm_type'

In [32]:
features_pd = pd.json_normalize(json_info["features"])

In [33]:
properties_df = features_pd[["properties.address_line1", "properties.address_line2", "properties.place_id"]]
properties_df

,properties.address_line1,properties.address_line2,properties.place_id
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...


In [34]:
# Rename columns
properties_df = properties_df.rename(columns = {"properties.address_line1": "Property", "properties.address_line2": "Address2", "properties.place_id": "Place_ID"})
properties_df

,Property,Address2,Place_ID
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...


In [35]:
# Create separate columns for the address fields
properties_df[['Address', 'City', 'State_Zip', 'Country']] = properties_df.Address2.str.split(",", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...,1633 Auburn Avenue,Cleveland,OH 44113,United States of America
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...,9308 Clifton Boulevard,Cleveland,OH 44102,United States of America
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...,East 9th Street,Cleveland,OH 44114,United States of America
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...,Selzer Way,Cleveland,OH 44114,United States of America
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...,8484 Madison Avenue,Cleveland,OH 44102,United States of America


In [36]:
properties_df = properties_df.dropna()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...,1633 Auburn Avenue,Cleveland,OH 44113,United States of America
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...,9308 Clifton Boulevard,Cleveland,OH 44102,United States of America
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...,East 9th Street,Cleveland,OH 44114,United States of America
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...,Selzer Way,Cleveland,OH 44114,United States of America
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...,8484 Madison Avenue,Cleveland,OH 44102,United States of America


In [37]:
properties_df["State_Zip"] = properties_df['State_Zip'].str.strip()
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...,1633 Auburn Avenue,Cleveland,OH 44113,United States of America
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...,9308 Clifton Boulevard,Cleveland,OH 44102,United States of America
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...,East 9th Street,Cleveland,OH 44114,United States of America
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...,Selzer Way,Cleveland,OH 44114,United States of America
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...,8484 Madison Avenue,Cleveland,OH 44102,United States of America


In [38]:
properties_df.State_Zip.str.split(" ", expand = True)

,0,1
0,OH,44114
1,OH,44114
2,OH,44109
3,OH,44113
4,OH,44115
5,OH,44113
6,OH,44102
7,OH,44114
8,OH,44114
9,OH,44102


In [39]:
properties_df[['State', 'Zip']] = properties_df.State_Zip.str.split(" ", expand = True)
properties_df

,Property,Address2,Place_ID,Address,City,State_Zip,Country,State,Zip
0,Great Lakes Science Center,"601 Erieside Avenue, Cleveland, OH 44114, Unit...",51512b54fd976c54c059bad57810f5c04440f00102f901...,601 Erieside Avenue,Cleveland,OH 44114,United States of America,OH,44114
1,International Women's Air and Space Museum,"1501 North Marginal Road, Cleveland, OH 44114,...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,1501 North Marginal Road,Cleveland,OH 44114,United States of America,OH,44114
2,A Christmas Story House Museum,"1103 Rowley Avenue, Cleveland, OH 44109, Unite...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...,1103 Rowley Avenue,Cleveland,OH 44109,United States of America,OH,44109
3,Frederich Bicycle,"3800 Lorain Avenue, Cleveland, OH 44113, Unite...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...,3800 Lorain Avenue,Cleveland,OH 44113,United States of America,OH,44113
4,Graffiti Caps,"3200 Carnegie Avenue, Cleveland, OH 44115, Uni...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...,3200 Carnegie Avenue,Cleveland,OH 44115,United States of America,OH,44115
5,Leavened,"1633 Auburn Avenue, Cleveland, OH 44113, Unite...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...,1633 Auburn Avenue,Cleveland,OH 44113,United States of America,OH,44113
6,Christophier Custom Clothing,"9308 Clifton Boulevard, Cleveland, OH 44102, U...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...,9308 Clifton Boulevard,Cleveland,OH 44102,United States of America,OH,44102
7,Rock and Roll Hall of Fame,"East 9th Street, Cleveland, OH 44114, United S...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...,East 9th Street,Cleveland,OH 44114,United States of America,OH,44114
8,Cleveland Hungarian Heritage Museum,"Selzer Way, Cleveland, OH 44114, United States...",516249b9fb1c6c54c0595546852fa7c04440f00103f901...,Selzer Way,Cleveland,OH 44114,United States of America,OH,44114
9,Family Dollar,"8484 Madison Avenue, Cleveland, OH 44102, Unit...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...,8484 Madison Avenue,Cleveland,OH 44102,United States of America,OH,44102


In [40]:
no_duplicates_df = properties_df.drop_duplicates(subset = "Place_ID", inplace = True)
no_duplicates_df

In [44]:
#create new dataframe for categories
categories2_df= pd.DataFrame(columns=['Category', 'Place_ID'])
categories2_df

,Category,Place_ID


In [45]:
#read json file and insert each line - see info again (delete later)
pprint(json_info)

{'features': [{'geometry': {'coordinates': [-81.69677670688294,
                                            41.50747877025738],
                            'type': 'Point'},
               'properties': {'address_line1': 'Great Lakes Science Center',
                              'address_line2': '601 Erieside Avenue, '
                                               'Cleveland, OH 44114, United '
                                               'States of America',
                              'categories': ['building',
                                             'building.tourism',
                                             'entertainment',
                                             'entertainment.museum',
                                             'internet_access',
                                             'wheelchair',
                                             'wheelchair.yes'],
                              'city': 'Cleveland',
                              'country': 

In [70]:
print(json_info["features"][1]["properties"]["categories"])
print(json_info["features"][1])
print(json_info["features"][1]["properties"]["place_id"])

['entertainment', 'entertainment.museum', 'no_fee', 'no_fee.no']
{'type': 'Feature', 'properties': {'name': "International Women's Air and Space Museum", 'country': 'United States', 'country_code': 'us', 'state': 'Ohio', 'county': 'Cuyahoga County', 'city': 'Cleveland', 'postcode': '44114', 'district': 'Downtown Cleveland', 'street': 'North Marginal Road', 'housenumber': '1501', 'lon': -81.6902994, 'lat': 41.5113986, 'state_code': 'OH', 'formatted': "International Women's Air and Space Museum, 1501 North Marginal Road, Cleveland, OH 44114, United States of America", 'address_line1': "International Women's Air and Space Museum", 'address_line2': '1501 North Marginal Road, Cleveland, OH 44114, United States of America', 'categories': ['entertainment', 'entertainment.museum', 'no_fee', 'no_fee.no'], 'details': ['details', 'details.contact', 'details.wiki_and_media'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Lice

In [89]:
#df2 = {'Name': 'Amy', 'Maths': 89, 'Science': 93} 
#df = df.append(df2, ignore_index = True)

category_table_df = pd.DataFrame(columns = ['Place_ID', 'Category'])

In [91]:
#import 
num_of_records = len(json_info['features'])

for i in range(num_of_records):
    place_id_var = json_info["features"][i]["properties"]["place_id"]

    for category in json_info["features"][i]["properties"]["categories"]:
        #print(j)
        new_row = {'Place_ID': place_id_var, 'Category': category}

        #https://stackoverflow.com/questions/75956209/error-dataframe-object-has-no-attribute-append
        category_table_df = category_table_df._append(new_row, ignore_index = True)


In [92]:
category_table_df

,Place_ID,Category
0,51512b54fd976c54c059bad57810f5c04440f00102f901...,building
1,51512b54fd976c54c059bad57810f5c04440f00102f901...,building.tourism
2,51512b54fd976c54c059bad57810f5c04440f00102f901...,entertainment
3,51512b54fd976c54c059bad57810f5c04440f00102f901...,entertainment.museum
4,51512b54fd976c54c059bad57810f5c04440f00102f901...,internet_access
5,51512b54fd976c54c059bad57810f5c04440f00102f901...,wheelchair
6,51512b54fd976c54c059bad57810f5c04440f00102f901...,wheelchair.yes
7,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,entertainment
8,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,entertainment.museum
9,51b2dc88dd2d6c54c059cf10648275c14440f00103f901...,no_fee


In [93]:
category_table_df_pl = pl.from_pandas(category_table_df)
category_table_df_pl

Place_ID,Category
str,str
"""51512b54fd976c…","""building"""
"""51512b54fd976c…","""building.touri…"
"""51512b54fd976c…","""entertainment"""
"""51512b54fd976c…","""entertainment.…"
"""51512b54fd976c…","""internet_acces…"
"""51512b54fd976c…","""wheelchair"""
"""51512b54fd976c…","""wheelchair.yes…"
"""51b2dc88dd2d6c…","""entertainment"""
"""51b2dc88dd2d6c…","""entertainment.…"


In [41]:
categories_df = features_pd[["properties.categories", "properties.place_id"]]
categories_df

,properties.categories,properties.place_id
0,"[building, building.tourism, entertainment, en...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,"[entertainment, entertainment.museum, no_fee, ...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,"[building, building.tourism, entertainment, en...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,"[commercial, commercial.outdoor_and_sport, com...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,"[commercial, commercial.clothing, commercial.c...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
5,"[commercial, commercial.food_and_drink, commer...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...
6,"[commercial, commercial.clothing, commercial.c...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...
7,"[building, building.tourism, entertainment, en...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...
8,"[entertainment, entertainment.museum]",516249b9fb1c6c54c0595546852fa7c04440f00103f901...
9,"[building, building.commercial, commercial, co...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...


In [42]:
# Rename columns
categories_df = categories_df.rename(columns = {"properties.categories": "Categories", "properties.place_id": "Place_ID"})
categories_df

,Categories,Place_ID
0,"[building, building.tourism, entertainment, en...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,"[entertainment, entertainment.museum, no_fee, ...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,"[building, building.tourism, entertainment, en...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,"[commercial, commercial.outdoor_and_sport, com...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,"[commercial, commercial.clothing, commercial.c...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
5,"[commercial, commercial.food_and_drink, commer...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...
6,"[commercial, commercial.clothing, commercial.c...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...
7,"[building, building.tourism, entertainment, en...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...
8,"[entertainment, entertainment.museum]",516249b9fb1c6c54c0595546852fa7c04440f00103f901...
9,"[building, building.commercial, commercial, co...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...


In [43]:
categories_df = categories_df.dropna()
categories_df

,Categories,Place_ID
0,"[building, building.tourism, entertainment, en...",51512b54fd976c54c059bad57810f5c04440f00102f901...
1,"[entertainment, entertainment.museum, no_fee, ...",51b2dc88dd2d6c54c059cf10648275c14440f00103f901...
2,"[building, building.tourism, entertainment, en...",5164ffa097066c54c0594f6fd80affbb4440f00102f901...
3,"[commercial, commercial.outdoor_and_sport, com...",5118963fdf966d54c059dfc9e4198fbd4440f00103f901...
4,"[commercial, commercial.clothing, commercial.c...",5151fb08579b6a54c059d240edeb0ac04440f00103f901...
5,"[commercial, commercial.food_and_drink, commer...",51fa4800ed7d6c54c059136ae905e9bc4440f00103f901...
6,"[commercial, commercial.clothing, commercial.c...",514459f8fada6f54c059d8a69a9e0bbe4440f00103f901...
7,"[building, building.tourism, entertainment, en...",5166b418a7826c54c059e0fb0f2d17c14440f00101f901...
8,"[entertainment, entertainment.museum]",516249b9fb1c6c54c0595546852fa7c04440f00103f901...
9,"[building, building.commercial, commercial, co...",51b6ea31aa8a6f54c0590388df0b22bd4440f00102f901...
